In [59]:
"""Created on Aug 9 12:38:36 2022

@author: Sophia Drozd
"""
import pandas as pd
import numpy as np
import rioxarray


In [60]:
ukr = rioxarray.open_rasterio("C:\\Users\\Sofi\\Downloads\\Ukraine_regions_sinu_500m.tif")

In [61]:
ukr_reg = ukr.squeeze().drop("spatial_ref").drop("band")
ukr_reg.name = "region"
res = ukr_reg.to_dataframe().reset_index()


In [62]:
res[['x','y']] = res[['x','y']].astype('float32')

In [63]:
ukr_mask = res[res['region']>0]

In [64]:
def prepare_crop(df,year):
    for i in range(5,9):
        crop = rioxarray.open_rasterio("C:\\Users\\Sofi\\Desktop\\"+str(i)+"\\"+str(i)+"_"+str(year)+"_new.tif").squeeze().drop("spatial_ref").drop("band")
        crop.name = "crop_"+str(i)+"_"+str(year)
        crop = crop.to_dataframe().reset_index()
        crop[['x','y']] = crop[['x','y']].astype('float32')
        df = df.merge(crop, on = ['x','y'])
    return df
        

In [65]:
merged = prepare_crop(ukr_mask,2021)

In [66]:
merged

,y,x,region,crop_5_2021,crop_6_2021,crop_7_2021,crop_8_2021
0,5824072.5,2253321.50,24,0,0,31,0
1,5823609.0,2252394.75,24,53,0,0,0
2,5823609.0,2252858.00,24,56,0,0,0
3,5823609.0,2253321.50,24,22,0,0,0
4,5823609.0,2253784.75,24,68,0,26,0
...,...,...,...,...,...,...,...
2791332,4935902.0,2681422.25,25,0,0,0,0
2791333,4935902.0,2683275.50,25,0,0,0,0
2791334,4935902.0,2683739.00,25,0,0,0,0
2791335,4935902.0,2684202.25,25,0,0,0,0


In [67]:
def merge(df, file):
    data = rioxarray.open_rasterio(file).squeeze().drop("spatial_ref").drop("band")
    data.name = file[29:-4]
    data = data.to_dataframe().reset_index()
    data[['x','y']]= data[['x','y']].astype("float32")
    for b in range(0,8):
        band = data[data.band==b].drop(['band'],axis=1)
        band = band.rename(columns = {file[29:-4] : file[29:-4]+"_"+str(b+1)})
        df = df.merge(band, on = ['x','y'])
    return df
    

In [68]:
import glob
modis_data = glob.glob("C:\\Users\\Sofi\\Downloads\\2021\\*.tif")

In [69]:
len(modis_data)

46

In [70]:
for file in modis_data:
    print(file)
    merged = merge(merged,file)

C:\Users\Sofi\Downloads\2021\MOD09A1.A2021001.Ukraine.brdf_product.02.02.tif
C:\Users\Sofi\Downloads\2021\MOD09A1.A2021009.Ukraine.brdf_product.02.02.tif
C:\Users\Sofi\Downloads\2021\MOD09A1.A2021017.Ukraine.brdf_product.02.02.tif
C:\Users\Sofi\Downloads\2021\MOD09A1.A2021025.Ukraine.brdf_product.02.02.tif
C:\Users\Sofi\Downloads\2021\MOD09A1.A2021033.Ukraine.brdf_product.02.02.tif
C:\Users\Sofi\Downloads\2021\MOD09A1.A2021041.Ukraine.brdf_product.02.02.tif
C:\Users\Sofi\Downloads\2021\MOD09A1.A2021049.Ukraine.brdf_product.02.02.tif
C:\Users\Sofi\Downloads\2021\MOD09A1.A2021057.Ukraine.brdf_product.02.02.tif
C:\Users\Sofi\Downloads\2021\MOD09A1.A2021065.Ukraine.brdf_product.02.02.tif
C:\Users\Sofi\Downloads\2021\MOD09A1.A2021073.Ukraine.brdf_product.02.02.tif
C:\Users\Sofi\Downloads\2021\MOD09A1.A2021081.Ukraine.brdf_product.02.02.tif
C:\Users\Sofi\Downloads\2021\MOD09A1.A2021089.Ukraine.brdf_product.02.02.tif
C:\Users\Sofi\Downloads\2021\MOD09A1.A2021097.Ukraine.brdf_product.02.02.tif

In [18]:
merged.dtypes

y                                                float32
x                                                float32
region                                             uint8
crop_5_2020                                        uint8
crop_6_2020                                        uint8
                                                  ...   
MOD09A1.A2020361.Ukraine.brdf_product.02.02_4      int16
MOD09A1.A2020361.Ukraine.brdf_product.02.02_5      int16
MOD09A1.A2020361.Ukraine.brdf_product.02.02_6      int16
MOD09A1.A2020361.Ukraine.brdf_product.02.02_7      int16
MOD09A1.A2020361.Ukraine.brdf_product.02.02_8      int16
Length: 375, dtype: object

In [71]:
merged.to_csv("data_2021.csv")